In [2]:
!pip install PyPDF2

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from PyPDF2 import PdfReader

import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.collocations import *

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/jeffrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jeffrey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/jeffrey/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeffrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
job_Descriptions = pd.read_csv('/Users/jeffrey/Desktop/jobs_data.csv')
print(job_Descriptions.head())

reader = PdfReader("/Users/jeffrey/Desktop/RandomResume2.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
resume = page.extract_text()
resume

                   Job Title              Salary Estimate  \
0             Data Scientist   $53K-$91K (Glassdoor est.)   
1  Healthcare Data Scientist  $63K-$112K (Glassdoor est.)   
2             Data Scientist   $80K-$90K (Glassdoor est.)   
3             Data Scientist   $56K-$97K (Glassdoor est.)   
4             Data Scientist  $86K-$143K (Glassdoor est.)   

                                     Job Description  Rating  \
0  Data Scientist\nLocation: Albuquerque, NM\nEdu...     3.8   
1  What You Will Do:\n\nI. General Summary\n\nThe...     3.4   
2  KnowBe4, Inc. is a high growth information sec...     4.8   
3  *Organization and Job ID**\nJob ID: 310709\n\n...     3.8   
4  Data Scientist\nAffinity Solutions / Marketing...     2.9   

                                 Company Name         Location  \
0                      Tecolote Research\n3.8  Albuquerque, NM   
1  University of Maryland Medical System\n3.4    Linthicum, MD   
2                                KnowBe4\n4.8   Cl

"Malik Rabb\nSeattle, WA | (123) 456-7891 | mrabb@email.com\nSummary\nData Scientist with strong math background and 3+ years of experience using predictive\nmodeling, data processing, and data mining algorithms to solve challenging business problems.\nInvolved in Python open source community and passionate about deep reinforcement learning.\nEducation\nCoral Springs University\nCurrent - Current\nBachelor of Science in Mathematics\nExperience\nRiver Tech, Data Scientist\nJul '19 - Current\n●\nBuilt fuzzy matching algorithm using k-nearest neighbors to identify non-exact matching\nduplicates\n●\nDesigned and developed real time recommendation engine to rank sales leads for upsell\nopportunities\n●\nRefined personalization algorithms for 1M+ customers on web and mobile\n●\nTransformed raw data into MySQL with custom-made ETL application to prepare unruly\ndata for machine learning\nRetail Ocean, Data Scientist\nAug '15 - Jul '19\n●\nLeveraged 200M+ tweets to develop sentiment analysis m

In [9]:
def preProcessing(resume):
    resume1 = resume.lower().replace("\n"," ") #rid of all nextLines
    resume2 = resume1.replace("●", "")  #rid of all bullet points
    return resume2

In [10]:
def tokenize(resume):
    tokens = word_tokenize(resume) 
    stop_words = set(stopwords.words('english') + list(punctuation))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [11]:
def lemmatize(tokenized_resume):
    lemmatizer = WordNetLemmatizer()
    processed_resume = [lemmatizer.lemmatize(word) for word in word_tokenize(" ".join(tokenized_resume))]
    return processed_resume

In [12]:
processed_resume = preProcessing(resume)
processed_resume = tokenize(processed_resume)
processed_resume = lemmatize(processed_resume)
processed_resume

['malik',
 'rabb',
 'seattle',
 'wa',
 '123',
 '456-7891',
 'mrabb',
 'email.com',
 'summary',
 'data',
 'scientist',
 'strong',
 'math',
 'background',
 '3+',
 'year',
 'experience',
 'using',
 'predictive',
 'modeling',
 'data',
 'processing',
 'data',
 'mining',
 'algorithm',
 'solve',
 'challenging',
 'business',
 'problem',
 'involved',
 'python',
 'open',
 'source',
 'community',
 'passionate',
 'deep',
 'reinforcement',
 'learning',
 'education',
 'coral',
 'spring',
 'university',
 'current',
 'current',
 'bachelor',
 'science',
 'mathematics',
 'experience',
 'river',
 'tech',
 'data',
 'scientist',
 'jul',
 "'19",
 'current',
 'built',
 'fuzzy',
 'matching',
 'algorithm',
 'using',
 'k-nearest',
 'neighbor',
 'identify',
 'non-exact',
 'matching',
 'duplicate',
 'designed',
 'developed',
 'real',
 'time',
 'recommendation',
 'engine',
 'rank',
 'sale',
 'lead',
 'upsell',
 'opportunity',
 'refined',
 'personalization',
 'algorithm',
 '1m+',
 'customer',
 'web',
 'mobile',
 't

In [13]:
def BiGrams(resume):
    finder = BigramCollocationFinder.from_words(resume)
    temp = []
    bigrams = []
    for i in sorted(finder.ngram_fd.items()):
        if i[1] >= 2:
            print(i)
            temp.append(i[0])
        else:
            pass

    for j in temp:
        bigrams.append(" ".join(j))
    return bigrams

In [14]:
bigrams_resume = BiGrams(processed_resume)
bigrams_resume

(('coral', 'spring'), 2)
(('data', 'mining'), 2)
(('data', 'scientist'), 3)
(('jul', "'19"), 2)
(('machine', 'learning'), 3)


['coral spring',
 'data mining',
 'data scientist',
 "jul '19",
 'machine learning']

In [15]:
resume_frame = pd.DataFrame()
resume_frame['job description'] = [""]
resume_frame['All'] = " ".join(processed_resume).join(bigrams_resume)

In [16]:
def reccomend(highest_CosSim, job_desc, scores):
    rec = pd.DataFrame(columns = ['Company Name',  'Job Title', 'Score', 'Job Description', 'Salary Estimate'])
    sc = 0
    for i in highest_CosSim:
        rec.at[sc, 'Company Name'] = job_Descriptions['Company Name'][i].replace("\n", "")
        rec.at[sc, 'Job Title'] = job_Descriptions['Job Title'][i]
        rec.at[sc, 'Score'] =  scores[sc]
        rec.at[sc, 'Job Description'] = job_Descriptions['Job Description'][i]
        rec.at[sc, 'Salary Estimate'] = job_Descriptions['Salary Estimate'][i]
        sc += 1
    return rec

In [17]:
def CosSim(job_desc, resume):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    # TF-IDF vectorize + Training the data
    tfidf_jobDesc = tfidf_vectorizer.fit_transform(job_desc)

    # TF-IDF vectorize 
    tfidf_Resume = tfidf_vectorizer.transform(resume)

    # Using cosine_similarity on (job_desc) & (resume)
    cos_sim_tfidf = map(lambda x: cosine_similarity(tfidf_Resume,x),tfidf_jobDesc)

    cos_sim = list(cos_sim_tfidf)
    return cos_sim
    
cos_sim = CosSim(job_Descriptions['Job Description'].drop_duplicates(), resume_frame['All'])
cos_sim

[array([[0.11002017]]),
 array([[0.11525895]]),
 array([[0.12556852]]),
 array([[0.09281272]]),
 array([[0.11427428]]),
 array([[0.09856593]]),
 array([[0.11124216]]),
 array([[0.13688187]]),
 array([[0.04996182]]),
 array([[0.08218644]]),
 array([[0.06159697]]),
 array([[0.05465006]]),
 array([[0.13477727]]),
 array([[0.12041641]]),
 array([[0.15603279]]),
 array([[0.07560513]]),
 array([[0.02008502]]),
 array([[0.11569006]]),
 array([[0.04850651]]),
 array([[0.07774014]]),
 array([[0.16595995]]),
 array([[0.10067104]]),
 array([[0.10714644]]),
 array([[0.12809637]]),
 array([[0.08865973]]),
 array([[0.06438056]]),
 array([[0.12885543]]),
 array([[0.0594426]]),
 array([[0.0720263]]),
 array([[0.11363058]]),
 array([[0.11178294]]),
 array([[0.07999765]]),
 array([[0.05837062]]),
 array([[0.12072859]]),
 array([[0.13845501]]),
 array([[0.09108365]]),
 array([[0.04626885]]),
 array([[0.10528884]]),
 array([[0.10986446]]),
 array([[0.06091194]]),
 array([[0.04332715]]),
 array([[0.0521476

In [18]:
def KNN(job_desc, resume):

    #Vectorize stopwords
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')

    #KNN Model 
    KNN = NearestNeighbors(n_neighbors=21, p=2) # of 463 unique values using eulcidean distance 

    # Train KNN Model
    KNN.fit(tfidf_vectorizer.fit_transform(job_desc))

    # Predict (Find Scores) KNN Model
    NNs = KNN.kneighbors(tfidf_vectorizer.transform(resume))

    return NNs

NNs = KNN(job_Descriptions['Job Description'].drop_duplicates(), resume_frame['All'])
NNs

(array([[1.27974474, 1.28551147, 1.28656921, 1.28702008, 1.29154176,
         1.2952    , 1.29657502, 1.29666145, 1.29856743, 1.29883212,
         1.29920531, 1.30226342, 1.30229496, 1.30419343, 1.30466027,
         1.30696574, 1.30960397, 1.3096494 , 1.30996995, 1.31019619,
         1.31100211]]),
 array([[ 48, 412,  69, 169,  20, 392,  55,  60,  71,  64,  14,  43, 231,
         389, 136, 115, 404, 385, 364, 221, 420]]))

In [19]:
highest_CosSim = sorted(range(len(cos_sim)), key=lambda i: cos_sim[i], reverse=True)[:20] #change amount of reccomendations 
scores = [cos_sim[i][0][0] for i in highest_CosSim]
top_cosSim=reccomend(highest_CosSim, job_Descriptions['Job Description'].drop_duplicates(), scores)
top_cosSim

,Company Name,Job Title,Score,Job Description,Salary Estimate
0,Centauri4.7,Spectral Scientist/Engineer,0.181127,Thank you for your interest in joining the Cen...,$56K-$117K (Glassdoor est.)
1,Applied Research Laboratories3.7,Geospatial Software Developer and Data Scientist,0.17373,Job Posting Title:\n\nGeospatial Software Deve...,$82K-$129K(Employer est.)
2,HP Inc.4.0,R&D Data Analysis Scientist,0.17237,Applies developed level of subject matter know...,$65K-$130K (Glassdoor est.)
3,OneMagnify4.3,Analytics Manager,0.17179,Responsible for both high level and granular m...,$59K-$116K (Glassdoor est.)
4,Porch3.2,Data Scientist,0.16596,About Our Team\n\nWe understand that the first...,$81K-$130K (Glassdoor est.)
5,Advanced BioScience Laboratories2.7,Staff Scientist- Upstream PD,0.161228,As a Staff Scientist in Upstream Process Devel...,$49K-$113K (Glassdoor est.)
6,Sartorius3.5,Data Scientist,0.159447,"Sartorius Stedim Data Analytics, a market lead...",$56K-$95K (Glassdoor est.)
7,The David J. Joseph Company4.4,Senior Data Scientist,0.159335,Overview\n\n\nEveryone wants to work with peop...,$82K-$132K (Glassdoor est.)
8,Grainger3.6,"Director, Data Science",0.156861,"Reporting to the Sr. Director, Analytics & Dat...",$139K-$220K (Glassdoor est.)
9,DICK'S Sporting Goods - Corporate3.8,Data Scientist,0.156518,Description\n\nThe core data science team at D...,$61K-$109K (Glassdoor est.)


In [20]:
closest_NN = NNs[1][0][1:]
scores = NNs[0][0][1:]
knn = reccomend(closest_NN, job_Descriptions, scores)
knn

,Company Name,Job Title,Score,Job Description,Salary Estimate
0,Applied Research Laboratories3.7,Geospatial Software Developer and Data Scientist,1.285511,Job Posting Title:\n\nGeospatial Software Deve...,$82K-$129K(Employer est.)
1,HP Inc.4.0,R&D Data Analysis Scientist,1.286569,Applies developed level of subject matter know...,$65K-$130K (Glassdoor est.)
2,OneMagnify4.3,Analytics Manager,1.28702,Responsible for both high level and granular m...,$59K-$116K (Glassdoor est.)
3,Porch3.2,Data Scientist,1.291542,About Our Team\n\nWe understand that the first...,$81K-$130K (Glassdoor est.)
4,Advanced BioScience Laboratories2.7,Staff Scientist- Upstream PD,1.2952,As a Staff Scientist in Upstream Process Devel...,$49K-$113K (Glassdoor est.)
5,Sartorius3.5,Data Scientist,1.296575,"Sartorius Stedim Data Analytics, a market lead...",$56K-$95K (Glassdoor est.)
6,The David J. Joseph Company4.4,Senior Data Scientist,1.296661,Overview\n\n\nEveryone wants to work with peop...,$82K-$132K (Glassdoor est.)
7,Grainger3.6,"Director, Data Science",1.298567,"Reporting to the Sr. Director, Analytics & Dat...",$139K-$220K (Glassdoor est.)
8,DICK'S Sporting Goods - Corporate3.8,Data Scientist,1.298832,Description\n\nThe core data science team at D...,$61K-$109K (Glassdoor est.)
9,Takeda Pharmaceuticals3.7,Data Scientist,1.299205,Job Description\n\n\nTakeda is looking for a D...,$83K-$144K (Glassdoor est.)


In [21]:
result = knn[['Job Description','Job Title', 'Company Name', 'Score']].merge(top_cosSim[['Job Description','Score']], on= "Job Description")
result = result.rename(columns={"Score_x": "KNN", "Score_y": "Cos Sim"})
result

,Job Description,Job Title,Company Name,KNN,Cos Sim
0,Job Posting Title:\n\nGeospatial Software Deve...,Geospatial Software Developer and Data Scientist,Applied Research Laboratories3.7,1.285511,0.17373
1,Applies developed level of subject matter know...,R&D Data Analysis Scientist,HP Inc.4.0,1.286569,0.17237
2,Responsible for both high level and granular m...,Analytics Manager,OneMagnify4.3,1.28702,0.17179
3,About Our Team\n\nWe understand that the first...,Data Scientist,Porch3.2,1.291542,0.16596
4,As a Staff Scientist in Upstream Process Devel...,Staff Scientist- Upstream PD,Advanced BioScience Laboratories2.7,1.2952,0.161228
5,"Sartorius Stedim Data Analytics, a market lead...",Data Scientist,Sartorius3.5,1.296575,0.159447
6,Overview\n\n\nEveryone wants to work with peop...,Senior Data Scientist,The David J. Joseph Company4.4,1.296661,0.159335
7,"Reporting to the Sr. Director, Analytics & Dat...","Director, Data Science",Grainger3.6,1.298567,0.156861
8,Description\n\nThe core data science team at D...,Data Scientist,DICK'S Sporting Goods - Corporate3.8,1.298832,0.156518
9,Job Description\n\n\nTakeda is looking for a D...,Data Scientist,Takeda Pharmaceuticals3.7,1.299205,0.156033


In [22]:
result['KNN'] = result['KNN']/2
result['Cos Sim'] = result['Cos Sim']/2
result['Final'] = result['KNN'] + result['Cos Sim']
result.sort_values(by = "Final", ascending = False)
result

,Job Description,Job Title,Company Name,KNN,Cos Sim,Final
0,Job Posting Title:\n\nGeospatial Software Deve...,Geospatial Software Developer and Data Scientist,Applied Research Laboratories3.7,0.642756,0.086865,0.729621
1,Applies developed level of subject matter know...,R&D Data Analysis Scientist,HP Inc.4.0,0.643285,0.086185,0.72947
2,Responsible for both high level and granular m...,Analytics Manager,OneMagnify4.3,0.64351,0.085895,0.729405
3,About Our Team\n\nWe understand that the first...,Data Scientist,Porch3.2,0.645771,0.08298,0.728751
4,As a Staff Scientist in Upstream Process Devel...,Staff Scientist- Upstream PD,Advanced BioScience Laboratories2.7,0.6476,0.080614,0.728214
5,"Sartorius Stedim Data Analytics, a market lead...",Data Scientist,Sartorius3.5,0.648288,0.079723,0.728011
6,Overview\n\n\nEveryone wants to work with peop...,Senior Data Scientist,The David J. Joseph Company4.4,0.648331,0.079667,0.727998
7,"Reporting to the Sr. Director, Analytics & Dat...","Director, Data Science",Grainger3.6,0.649284,0.078431,0.727714
8,Description\n\nThe core data science team at D...,Data Scientist,DICK'S Sporting Goods - Corporate3.8,0.649416,0.078259,0.727675
9,Job Description\n\n\nTakeda is looking for a D...,Data Scientist,Takeda Pharmaceuticals3.7,0.649603,0.078016,0.727619
